# Поробуем строить ембединги по айтемам

In [1]:
import os
import csv
import json
import random
import pickle
import glob
from datetime import datetime
from collections import defaultdict

import gensim
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

plt.style.use('seaborn-poster')

%matplotlib inline

## Word2Vec gensim пробно 1 чанк

In [2]:
    %%time
    train_list_products = []
    valid_time = datetime.strptime('2019-03-01 00:00:00', '%Y-%m-%d %H:%M:%S')
    chunk_file = '../_processed_data/client_tr_history_1.tsv'
    with open(chunk_file, 'r') as chunk:
        for row in tqdm(chunk):
            client_id, transaction_history = row.split('\t')
            client_id, transaction_history = json.loads(client_id), json.loads(transaction_history)
            
            # tr history: dict -> list and sort it
            tr_history = [transaction_history[tr] for tr in transaction_history\
                          if datetime.strptime(
                                transaction_history[tr]['datetime'], '%Y-%m-%d %H:%M:%S'
                             ) < valid_time]
            sorted_transactions = sorted(tr_history, 
                                         key=lambda x: datetime.strptime(x['datetime'], '%Y-%m-%d %H:%M:%S'))
            cur_products = []
            for transaction in sorted_transactions:
                for pr in transaction['products']:
                    cur_products.append(pr['product_id'])
                    
            train_list_products.append(cur_products)

20000it [00:06, 3172.93it/s]

CPU times: user 6.49 s, sys: 118 ms, total: 6.61 s
Wall time: 6.31 s


In [10]:
train_list_products[0]

['95744496d2',
 'f36b7a80ae',
 'ad865591c6',
 '53fc95e177',
 '1c257c1a1b',
 '3981c51990',
 '439498bce2',
 '8610cef2f8',
 '93fcd1f96c',
 '2020b2edc3',
 '98528c8b99',
 '3b12a1b480',
 '1c3dd17f02',
 '381ae501bd',
 '53fc95e177',
 'fed2d0859c',
 '5d03ab4a61',
 '3b12a1b480',
 '95744496d2',
 'b54e01c5d1',
 'de25bccdaf',
 '0e571ffd73',
 '4dcf79043e',
 'de25bccdaf',
 '4dcf79043e',
 '1da9db7b6f',
 '9bf90feeb1',
 '3b12a1b480',
 '53fc95e177',
 '3b12a1b480',
 '53fc95e177',
 'c30a10c8a0',
 'bf07df54e1',
 '1e6fc59101',
 '8a35b897d9',
 '95744496d2',
 'e6a5597d19',
 '95744496d2',
 'bf07df54e1',
 '95744496d2',
 '2020b2edc3',
 '53fc95e177',
 '1c257c1a1b',
 '67d5dbb998',
 'b2110488a0',
 '4dcf79043e',
 'c6a848c19d',
 '439498bce2',
 'e2d1cbd093',
 'b2110488a0',
 '3b12a1b480',
 '53fc95e177',
 '95744496d2',
 '9842819ad3',
 '4a40dfd44d',
 '7d3309414b',
 '2954b43ed3',
 'c6a848c19d',
 '53fc95e177',
 'a49e3cddde',
 '2020b2edc3',
 '53fc95e177',
 '3b12a1b480',
 '7d3309414b',
 '47de7380af',
 '1da9db7b6f',
 'a265b81c

In [3]:
%%time
model = gensim.models.Word2Vec(sentences=train_list_products, seed=42, workers=8, iter=5,
                               size=100, window=10, min_count=2)

CPU times: user 20.4 s, sys: 56.9 ms, total: 20.4 s
Wall time: 7.43 s


In [5]:
model.wv['3e0d2b086a']

array([-0.42889738, -0.86854553, -0.89940727, -0.8449513 , -0.3571108 ,
       -0.5806324 , -0.4676343 ,  0.11515905,  0.97976846, -1.5146205 ,
       -2.176941  , -1.0826755 , -1.1259253 ,  1.9315263 ,  0.1561668 ,
        0.17985554,  1.5540669 ,  0.9042437 ,  1.9364153 ,  1.6636288 ,
        0.1985477 ,  1.4843434 , -0.52071327, -0.75854594, -1.5988696 ,
       -0.30155778,  0.4098121 ,  0.61005396,  1.1770308 , -0.33611986,
        0.07405066, -1.663303  ,  0.0782939 , -1.3812988 ,  0.6521126 ,
       -1.1044441 ,  1.8715409 , -0.5229926 ,  2.1231904 ,  0.9073692 ,
        2.1343095 ,  0.24298452,  1.7407728 ,  1.7985804 ,  0.6712736 ,
       -2.1391008 ,  0.90344286,  1.1347483 , -0.31753382, -0.73656577,
       -0.765492  , -1.3674322 , -1.8753607 , -0.84577036, -1.8116083 ,
        0.17416103,  0.5297875 ,  1.0539384 , -0.4732205 ,  0.05437944,
        1.1265175 , -1.1234438 , -0.30662656, -1.5135238 ,  2.1906188 ,
       -1.0397863 ,  1.9578285 , -1.8510605 ,  0.6021404 ,  2.76

In [9]:
model.wv.get_vector('jghj')

KeyError: "word 'jghj' not in vocabulary"

In [4]:
products = ['3e0d2b086a', 'eb43dae31f','f0b47b5188']
model.wv.most_similar(positive=products, topn=30)

[('d398d5def5', 0.9935541152954102),
 ('85c81fad32', 0.9934086799621582),
 ('95744496d2', 0.9919785857200623),
 ('d074c96d42', 0.9914391040802002),
 ('072baddbd3', 0.9910869598388672),
 ('c7c4b267ca', 0.9905596971511841),
 ('27ffa361d6', 0.9903650879859924),
 ('8ea0c2c897', 0.9901469349861145),
 ('2fb6b34fae', 0.9900996685028076),
 ('dce037852a', 0.9896588921546936),
 ('aaff89b3c9', 0.9894154667854309),
 ('3337c679b9', 0.9887217283248901),
 ('6373e1ed1d', 0.9885915517807007),
 ('28b7c31c56', 0.9880638718605042),
 ('d55c0d8ba6', 0.9880450367927551),
 ('f9a5ea37f4', 0.9878149628639221),
 ('dc9b9de8fb', 0.9875327348709106),
 ('514dae85f0', 0.987369179725647),
 ('492abc2587', 0.9865107536315918),
 ('1a1276d65e', 0.9864274263381958),
 ('03dfe29922', 0.9862962961196899),
 ('db598268d8', 0.9862934350967407),
 ('9842819ad3', 0.9859880208969116),
 ('9e8d780dfa', 0.9859122037887573),
 ('f4f8a33fdd', 0.9858095645904541),
 ('c341674ed6', 0.9855425357818604),
 ('a07e895d21', 0.9854148626327515),
 (

In [13]:
model.wv.most_similar('95744496d2')

[('3e0d2b086a', 0.9872453212738037),
 ('85c81fad32', 0.985153317451477),
 ('91aa6e7307', 0.9836788177490234),
 ('f9a5ea37f4', 0.9834887981414795),
 ('eb43dae31f', 0.9834136962890625),
 ('c7c4b267ca', 0.9832232594490051),
 ('492abc2587', 0.9830978512763977),
 ('8ea0c2c897', 0.9828910827636719),
 ('28b7c31c56', 0.9828265905380249),
 ('f0b47b5188', 0.9825232028961182)]

In [1]:
['w2v_' + str(i) for i in range(50)]

['w2v_0',
 'w2v_1',
 'w2v_2',
 'w2v_3',
 'w2v_4',
 'w2v_5',
 'w2v_6',
 'w2v_7',
 'w2v_8',
 'w2v_9',
 'w2v_10',
 'w2v_11',
 'w2v_12',
 'w2v_13',
 'w2v_14',
 'w2v_15',
 'w2v_16',
 'w2v_17',
 'w2v_18',
 'w2v_19',
 'w2v_20',
 'w2v_21',
 'w2v_22',
 'w2v_23',
 'w2v_24',
 'w2v_25',
 'w2v_26',
 'w2v_27',
 'w2v_28',
 'w2v_29',
 'w2v_30',
 'w2v_31',
 'w2v_32',
 'w2v_33',
 'w2v_34',
 'w2v_35',
 'w2v_36',
 'w2v_37',
 'w2v_38',
 'w2v_39',
 'w2v_40',
 'w2v_41',
 'w2v_42',
 'w2v_43',
 'w2v_44',
 'w2v_45',
 'w2v_46',
 'w2v_47',
 'w2v_48',
 'w2v_49']

## Word2vec все чанки

In [2]:
valid_time = datetime.strptime('2019-03-01 00:00:00', '%Y-%m-%d %H:%M:%S')

class TransactionDocCorpus:
    """
    Iterator over all chunks
    1 transaction = 1 document
    """
    def __init__(self, chunks_path_list, valid_time):
        self.chunks_path_list = chunks_path_list
        self.valid_time = valid_time
        
    def __iter__(self):
        for chunk_file in self.chunks_path_list:
            print(chunk_file)
            with open(chunk_file, 'r') as chunk:
                for row in tqdm(chunk):
                    client_id, transaction_history = row.split('\t')
                    client_id, transaction_history = json.loads(client_id), json.loads(transaction_history)
                    tr_history = [transaction_history[tr] for tr in transaction_history\
                                  if datetime.strptime(
                                    transaction_history[tr]['datetime'], '%Y-%m-%d %H:%M:%S'
                                     ) < valid_time]
                    sorted_transactions = sorted(tr_history, 
                                         key=lambda x: datetime.strptime(x['datetime'], '%Y-%m-%d %H:%M:%S'))
                    for transaction in sorted_transactions:
                        cur_products = []
                        for pr in transaction['products']:
                            cur_products.append(pr['product_id'])

                        yield cur_products

class ClientDocCorpus:
    """
    Iterator over all chunks
    1 client transaction_history = 1 document
    """
    def __init__(self, chunks_path_list, valid_time):
        self.chunks_path_list = chunks_path_list
        self.valid_time = valid_time
        
    def __iter__(self):
        for chunk_file in self.chunks_path_list:
            print(chunk_file)
            with open(chunk_file, 'r') as chunk:
                for row in tqdm(chunk):
                    client_id, transaction_history = row.split('\t')
                    client_id, transaction_history = json.loads(client_id), json.loads(transaction_history)
                    
                    tr_history = [transaction_history[tr] for tr in transaction_history\
                                  if datetime.strptime(
                                    transaction_history[tr]['datetime'], '%Y-%m-%d %H:%M:%S'
                                     ) < valid_time]
                    sorted_transactions = sorted(tr_history, 
                                         key=lambda x: datetime.strptime(x['datetime'], '%Y-%m-%d %H:%M:%S'))
                    cur_products = []
                    for transaction in sorted_transactions:
                        for pr in transaction['products']:
                            cur_products.append(pr['product_id'])

                    yield cur_products

In [3]:
valid_chunks = [
    '../_processed_data/client_tr_history_4.tsv', 
    '../_processed_data/client_tr_history_0.tsv',
    '../_processed_data/client_tr_history_15.tsv', 
    '../_processed_data/client_tr_history_16.tsv', 
    '../_processed_data/client_tr_history_6.tsv'
]
train_chunks = sorted(glob.glob('../_processed_data/client_tr_history_*'))
train_chunks = [chunk for chunk in train_chunks
                if chunk not in valid_chunks and chunk != '../_processed_data/client_tr_history_20.tsv']

In [4]:
%%time
transaction_level = TransactionDocCorpus(train_chunks, valid_time)
model1 = gensim.models.Word2Vec(sentences=transaction_level, seed=42, workers=8, iter=5,
                                size=50, window=10, min_count=2)

280it [00:00, 2781.53it/s]

../_processed_data/client_tr_history_1.tsv


20000it [00:06, 3048.44it/s]
575it [00:00, 2903.53it/s]

../_processed_data/client_tr_history_10.tsv


20000it [00:06, 3069.40it/s]
595it [00:00, 2935.84it/s]

...

../_processed_data/client_tr_history_8.tsv


20000it [00:07, 2625.67it/s]
595it [00:00, 2970.84it/s]

../_processed_data/client_tr_history_9.tsv


20000it [00:07, 2650.38it/s]

CPU times: user 14min 39s, sys: 8.79 s, total: 14min 48s
Wall time: 11min 24s


In [5]:
model1.save('../_model_files/w2v_01_26_1.wv')

In [6]:
%%time
transaction_level = ClientDocCorpus(train_chunks, valid_time)
model2 = gensim.models.Word2Vec(sentences=transaction_level, seed=42, workers=8, iter=5,
                                size=50, window=10, min_count=2)

273it [00:00, 2721.11it/s]

../_processed_data/client_tr_history_1.tsv


20000it [00:06, 3041.85it/s]
303it [00:00, 3015.09it/s]

../_processed_data/client_tr_history_10.tsv


20000it [00:06, 3035.45it/s]
588it [00:00, 2913.85it/s]

...

../_processed_data/client_tr_history_8.tsv


20000it [00:12, 1623.18it/s]
261it [00:00, 2603.35it/s]

../_processed_data/client_tr_history_9.tsv


20000it [00:09, 2005.64it/s]

CPU times: user 18min 10s, sys: 9.69 s, total: 18min 20s
Wall time: 14min 19s


In [7]:
model2.save('../_model_files/w2v_01_26_2.wv')